# NYC Food Rescue Optimization - Visualizations

Run the Julia export code first, then run this notebook to generate:
- Interactive Pareto curve (cost vs. equity)
- Network flow maps
- Demand heatmaps

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import folium
from folium.plugins import MarkerCluster
from IPython.display import display, HTML

## Load Data

In [2]:
pareto_df = pd.read_csv('viz_data/pareto_results.csv')
restaurants_df = pd.read_csv('viz_data/restaurants.csv')
centers_df = pd.read_csv('viz_data/donation_centers.csv')
neighborhoods_df = pd.read_csv('viz_data/neighborhoods.csv')

try:
    flows_df = pd.read_csv('viz_data/flows_balanced.csv')
except:
    flows_df = None

print(f"Pareto solutions: {len(pareto_df)}")
print(f"Restaurants: {len(restaurants_df)}")
print(f"Centers: {len(centers_df)}")
print(f"Neighborhoods: {len(neighborhoods_df)}")

Pareto solutions: 19
Restaurants: 319
Centers: 201
Neighborhoods: 591


## 1. Pareto Curve (Cost vs. Equity)

In [3]:
COLORS = {
    'background': '#0a0a0f',
    'surface': '#14141f',
    'primary': '#ff6b35',
    'secondary': '#4ecdc4',
    'accent': '#ffe66d',
    'text': '#f7f7f7',
    'text_muted': '#8b8b9e',
    'grid': '#2a2a3e',
}

df = pareto_df.sort_values('transport_cost').copy()

fig = go.Figure()

# Pareto line
fig.add_trace(go.Scatter(
    x=df['transport_cost'], y=df['equity_t'],
    mode='lines',
    line=dict(color=COLORS['primary'], width=3, shape='spline', smoothing=0.8),
    hoverinfo='skip'
))

# Points with hover info
hover_text = [f"<b>w_cost={row.w_cost}, w_eq={row.w_eq}</b><br>" +
              f"Transport: {row.transport_cost:,.0f}<br>" +
              f"Worst Unmet: {row.equity_t:,.0f}<br>" +
              f"Total Delivered: {row.total_recv:,.0f}"
              for _, row in df.iterrows()]

fig.add_trace(go.Scatter(
    x=df['transport_cost'], y=df['equity_t'],
    mode='markers',
    marker=dict(size=12, color=COLORS['secondary'], line=dict(color='white', width=2)),
    text=hover_text,
    hovertemplate='%{text}<extra></extra>'
))

# Annotations
cost_opt = df.loc[df['transport_cost'].idxmin()]
eq_opt = df.loc[df['equity_t'].idxmin()]

fig.add_annotation(x=cost_opt['transport_cost'], y=cost_opt['equity_t'],
                   text="Cost Optimal", showarrow=True, arrowcolor=COLORS['secondary'],
                   font=dict(color=COLORS['secondary']), ax=-50, ay=-30)
fig.add_annotation(x=eq_opt['transport_cost'], y=eq_opt['equity_t'],
                   text="Equity Optimal", showarrow=True, arrowcolor=COLORS['accent'],
                   font=dict(color=COLORS['accent']), ax=50, ay=30)

fig.update_layout(
    title=dict(text="<b>Pareto Frontier: Cost vs. Equity</b>", x=0.5, font=dict(size=20, color=COLORS['text'])),
    xaxis=dict(title="Transportation Cost", tickformat=',', gridcolor=COLORS['grid'], color=COLORS['text_muted']),
    yaxis=dict(title="Worst Unmet Demand", tickformat=',', gridcolor=COLORS['grid'], color=COLORS['text_muted']),
    plot_bgcolor=COLORS['surface'],
    paper_bgcolor=COLORS['background'],
    font=dict(color=COLORS['text']),
    showlegend=False,
    height=500
)

fig.show()

## 2. Network Map (Folium)

In [4]:
m = folium.Map(location=[40.7128, -73.9560], zoom_start=11, tiles='CartoDB dark_matter')

# Restaurants
restaurant_cluster = MarkerCluster(name='Restaurants')
for _, row in restaurants_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=4, color='#ff6b35', fill=True, fill_opacity=0.7,
        popup=f"Restaurant {int(row['id'])}<br>Supply: {row['supply']:,.0f}"
    ).add_to(restaurant_cluster)
restaurant_cluster.add_to(m)

# Centers
for _, row in centers_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        icon=folium.Icon(color='green', icon='archive', prefix='fa'),
        popup=f"Center {int(row['id'])}"
    ).add_to(m)

# Neighborhoods with demand
demand_nbhd = neighborhoods_df[neighborhoods_df['demand'] > 0]
max_demand = demand_nbhd['demand'].max()
for _, row in demand_nbhd.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3 + 12 * (row['demand'] / max_demand),
        color='#a855f7', fill=True, fill_opacity=0.5,
        popup=f"Neighborhood {int(row['id'])}<br>Demand: {row['demand']:,.0f}"
    ).add_to(m)

# Flows
if flows_df is not None:
    rest_coords = dict(zip(restaurants_df['id'], zip(restaurants_df['latitude'], restaurants_df['longitude'])))
    center_coords = dict(zip(centers_df['id'], zip(centers_df['latitude'], centers_df['longitude'])))
    nbhd_coords = dict(zip(neighborhoods_df['id'], zip(neighborhoods_df['latitude'], neighborhoods_df['longitude'])))
    
    sig_flows = flows_df[flows_df['flow'] > 1000]
    max_flow = sig_flows['flow'].max() if len(sig_flows) > 0 else 1
    
    for _, row in sig_flows.iterrows():
        from_coord = rest_coords.get(row['from_id']) if row['from_type'] == 'restaurant' else center_coords.get(row['from_id'])
        to_coord = center_coords.get(row['to_id']) if row['to_type'] == 'center' else nbhd_coords.get(row['to_id'])
        if from_coord and to_coord:
            folium.PolyLine(
                [from_coord, to_coord],
                weight=1 + 3 * (row['flow'] / max_flow),
                color='#ffe66d', opacity=0.6
            ).add_to(m)

folium.LayerControl().add_to(m)
m

## 3. Demand Heatmap

In [5]:
fig = go.Figure(go.Densitymapbox(
    lat=neighborhoods_df['latitude'],
    lon=neighborhoods_df['longitude'],
    z=neighborhoods_df['demand'],
    radius=20,
    colorscale=[
        [0, 'rgba(78, 205, 196, 0.1)'],
        [0.5, 'rgba(255, 107, 53, 0.6)'],
        [1, 'rgba(168, 85, 247, 1)']
    ],
    colorbar=dict(title=dict(text='Demand (lbs)'))
))

fig.update_layout(
    mapbox=dict(style='carto-darkmatter', center=dict(lat=40.7128, lon=-73.9560), zoom=10),
    title=dict(text="<b>Food Demand by Neighborhood</b>", x=0.5, font=dict(size=18, color='white')),
    paper_bgcolor='#0a0a0f',
    margin=dict(l=0, r=0, t=50, b=0),
    height=500
)

fig.show()

C:\Users\orsot\AppData\Local\Temp\ipykernel_44372\448637592.py:1: DeprecationWarning:

*densitymapbox* is deprecated! Use *densitymap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



## 4. Results Summary

In [6]:
total_supply = restaurants_df['supply'].sum()
total_demand = neighborhoods_df['demand'].sum()
best_equity = pareto_df['equity_t'].min()
best_cost = pareto_df['transport_cost'].min()

print(f"Total Supply: {total_supply:,.0f} lbs")
print(f"Total Demand: {total_demand:,.0f} lbs")
print(f"Supply/Demand Ratio: {100*total_supply/total_demand:.1f}%")
print(f"\nBest Transport Cost: {best_cost:,.0f}")
print(f"Best Equity (min worst unmet): {best_equity:,.0f}")
print(f"\nPareto Solutions: {len(pareto_df)}")

Total Supply: 16,034,410 lbs
Total Demand: 152,439,330 lbs
Supply/Demand Ratio: 10.5%

Best Transport Cost: 20,793,474,695,622,208
Best Equity (min worst unmet): 2,095,261

Pareto Solutions: 19
